In [1]:
import numpy as np

origindata = np.load('../data/dataset/origindata.npy')
meldata = np.load('../data/dataset/meldata.npy')
cqtdata = np.load('../data/dataset/cqtdata.npy')
stftdata = np.load('../data/dataset/stftdata.npy')
genedata = np.load('../data/dataset/genedata.npy')
label = np.load('../data/dataset/label.npy')
print(origindata.shape, meldata.shape, cqtdata.shape, stftdata.shape, genedata.shape, label.shape)

(7648, 132300) (7648, 258, 128) (7648, 258, 84) (7648, 258, 1025) (7648, 60, 12) (7648,)


In [9]:
import tensorflow as tf
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn import metrics
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

class Dataloader():
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.pointer = 0
        self.num_batch = 0

    # Shuffle the data
    def Shuffle(self, datalength):
        shuffle_indices = np.random.permutation(np.arange(datalength))
        return shuffle_indices

    def SplitBatches(self, data):
        datas = data[:self.num_batch * self.batch_size]
        data_batches = np.split(datas, self.num_batch, 0)
        return data_batches
    
    def load_data(self, x, y):
        self.x = np.array(x, dtype=np.float32)
        self.y = np.array(y, dtype=np.float32)

        # Shuffle the data
        shuffle_indices = self.Shuffle(self.x.shape[0])
        self.x = self.x[shuffle_indices]
        self.y = self.y[shuffle_indices]
        # Split batches
        self.num_batch = int(self.x.shape[0] / self.batch_size)
        self.pointer = 0

        self.x_batches = self.SplitBatches(self.x)
        self.y_batches = self.SplitBatches(self.y)

    def next_batch(self):
        x_batch = self.x_batches[self.pointer]
        y_batch = self.y_batches[self.pointer]

        self.pointer = (self.pointer + 1) % self.num_batch
        return x_batch, y_batch

    def reset_pointer(self):
        self.pointer = 0

# data sample
def sample(datax, datay):
    data0 = datax[datay == 0]
    data1 = datax[datay == 1]
    index = np.random.randint(low=0, high = data0.shape[0], size = data1.shape[0] * 2)
    data0 = data0[index]
    datax = np.concatenate([data1, data0], axis=0)
    datay = np.array([0] * datax.shape[0])
    datay[0:data1.shape[0]] = 1
    shuffle_indices = np.random.permutation(np.arange(datax.shape[0]))
    datax, datay = datax[shuffle_indices], datay[shuffle_indices]
    trainx, trainy = datax[:int(datax.shape[0] * 0.8)], datay[:int(datax.shape[0] * 0.8)]
    testx, testy = datax[int(datax.shape[0] * 0.8):], datay[int(datax.shape[0] * 0.8):]
    
    return trainx, trainy, testx, testy

# tslearn: euclidean, DTW, shaplets
def TSlearn(datax, datay, method='euclidean'):
    trainx, trainy, testx, testy = sample(datax, datay)
    print(datax.shape, datay.shape, trainx.shape, testx.shape)
    clf = KNeighborsTimeSeriesClassifier(n_neighbors=5, metric=method)
    clf.fit(trainx, trainy)
    y_pred = clf.predict(testx)

    print ('Precision: {:04f}\tRecall: {:04f}\tF1: {:04f}'.format(
        metrics.precision_score(testy, y_pred), 
        metrics.recall_score(testy, y_pred), 
        metrics.f1_score(testy, y_pred)))

# rnn: LSTM, GRU
def RNNlearn(datax, datay, method='LSTM', hiddens=[1024, 256]):
    trainx, trainy, testx, testy = sample(datax, datay)
    print(datax.shape, datay.shape, trainx.shape, testx.shape)
    
    # network
    tf.reset_default_graph()
    x = tf.placeholder(tf.float32, [None, datax.shape[1], datax.shape[2]], name='input_x')
    y = tf.placeholder(tf.float32, [None, 2], name='input_y')
    if method == 'LSTM':
        rnn_fw_cell = tf.contrib.rnn.BasicLSTMCell(num_units=hiddens[0])
        rnn_bw_cell = tf.contrib.rnn.BasicLSTMCell(num_units=hiddens[0])
    elif method == 'GRU':
        rnn_fw_cell = tf.contrib.rnn.GRUCell(num_units=hiddens[0])
        rnn_bw_cell = tf.contrib.rnn.GRUCell(num_units=hiddens[0])
        
    outputs_fw, _ = tf.nn.dynamic_rnn(rnn_fw_cell, x[:, :int(datax.shape[1] * 2 / 3)], initial_state=None, dtype=tf.float32, time_major=True, scope='fw')
    outputs_bw, _ = tf.nn.dynamic_rnn(rnn_bw_cell, x[:, int(datax.shape[1] * 2 / 3):][:, ::-1], initial_state=None, dtype=tf.float32, time_major=True, scope='bw')
    
    logits = tf.concat([outputs_fw[:, -1, :], outputs_bw[:, -1, :]], axis=-1)
    net = tf.layers.dense(logits, hiddens[1], activation=tf.nn.relu)
    out_logits = tf.layers.dense(net, 2)
    out = tf.nn.softmax(out_logits, name='output')
    
    loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=y, logits=out_logits))
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        optim = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.7).minimize(loss)
    
    # train
    trainy = np.eye(2, dtype=np.float32)[trainy]
#     trainloader = Dataloader(100)
#     trainloader.load_data(trainx, trainy)
#     testloader = Dataloader(100)
#     testloader.load_data(testx, testy)
    
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.)
    config = tf.ConfigProto(gpu_options=gpu_options)
    bestp = 0.0
    saver = tf.train.Saver()
    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(100):
            y_pred = []
            losses = []
            _, loss_ = sess.run([optim, loss], feed_dict={x: trainx, y: trainy})
            out_ = sess.run(out, feed_dict={x: testx})
            y_pred = np.argmax(out_, axis=1)
#             trainloader.reset_pointer()
#             for _ in range(trainloader.num_batch):
#                 x_batch, y_batch = trainloader.next_batch()
#                 _, loss_ = sess.run([optim, loss], feed_dict={x: x_batch, y: y_batch})
#                 losses.append(loss_)
                
#             testloader.reset_pointer()
#             for _ in range(testloader.num_batch):
#                 x_batch, y_batch = testloader.next_batch()
#                 out_ = sess.run(out, feed_dict={x: testx})
#                 y_pred.append(out_)
#             y_pred = np.argmax(y_pred)
            precision = metrics.precision_score(testy, y_pred)
            recall = metrics.recall_score(testy, y_pred)
            f1 = metrics.f1_score(testy, y_pred)
            if f1 > bestp:
                saver.save(sess, './model/rnn')
                bestp = f1
            print ('Epochs {:d}\tloss: {:04f}\tPrecision: {:04f}\tRecall: {:04f}\tF1: {:04f}'.format(i, loss_,
                precision, recall, f1))

In [13]:
# euclidean
TSlearn(origindata[:, range(0, origindata.shape[1], 100)], label, method='euclidean')

(7648, 1323) (7648,) (573, 1323) (144, 1323)
Precision: 0.235294	Recall: 0.102564	F1: 0.142857


In [10]:
RNNlearn(genedata, label, method='GRU', hiddens=[128, 64])

(7648, 60, 12) (7648,) (573, 60, 12) (144, 60, 12)


/home/hwj/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/hwj/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epochs 0	loss: 0.705520	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 1	loss: 0.681564	Precision: 0.323529	Recall: 0.936170	F1: 0.480874
Epochs 2	loss: 0.718564	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 3	loss: 0.642847	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 4	loss: 0.731084	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 5	loss: 0.652996	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 6	loss: 0.644079	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 7	loss: 0.640774	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 8	loss: 0.632796	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 9	loss: 0.628488	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 10	loss: 0.624584	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 11	loss: 0.617431	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 12	loss: 0.610248	Precision: 0.000000	Recall: 0.000000	F1: 0.000000
Epochs 13	loss: 0.608820	Precision: